In [4]:
from tqdm import tqdm
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from utils import construct_tools
from langgraph.prebuilt import ToolNode
import getpass
import os
from langgraph.graph.graph import CompiledGraph
from utils import process_dataset
from datasets import load_dataset

from dotenv import find_dotenv, load_dotenv


_ = load_dotenv(find_dotenv())


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("LANGCHAIN_API_KEY")


system_prompt = ("Answer the following question. "
                 "Remember your FINAL ANSWER should be clear and concise.(a single number or phrases, not a sentence!)"
                 "Follow the format: \n"
                 "FINAL ANSWER: <your answer>")



tools = construct_tools()
tool_node = ToolNode(tools)

In [5]:
async def react(item: dict, reactor: CompiledGraph) -> str:
    try:
        messages = [HumanMessage(content=item["context"] + item["question"])]
        input = {"messages": messages}
    except Exception:
        raise ValueError("Invalid dataset_name")
    
    try:
        return await reactor.ainvoke(input=input)   
    except Exception as e:
        print(f"Error: {e}")
        return str(e)

# gsm8k svamp tabmwp trivia_qa ambig_qa

In [1]:
dataset_name = "tabmwp"
mode = "react"
num_test_sample = 1000
temperature = 0
top_p = 1
batch_size = 100
version = "v2"
llm = ChatOpenAI(temperature=temperature, top_p=top_p, model="gpt-4o-mini", base_url="https://api.chsdw.top/v1", max_retries=3)
graph = create_react_agent(model=llm, tools=tools, state_modifier=system_prompt)

dataset = process_dataset(file_path=f"../data/{dataset_name}.jsonl")
if dataset_name == "ambig_qa":
    dataset = load_dataset("json", data_files=f"/Users/ariete/Projects/self-improve/output/inference/ambig_qa/cot_1000_temperature_0_top-p_1.jsonl", split="train")
if num_test_sample > 0:
    dataset = dataset.select(range(num_test_sample))
print(dataset, json.dumps({key: dataset[0][key] for key in ["context", "question", "answer"]}, indent=4))
results = []
print(llm.temperature, llm.top_p)

NameError: name 'ChatOpenAI' is not defined

In [6]:
import asyncio
for i in tqdm(range(0, len(dataset), batch_size)):
    batch = dataset.select(range(i, min(i+batch_size, len(dataset))))
    results.extend(await asyncio.gather(*(react(item=item, reactor=graph) for item in batch)))
    with open(f"/Users/ariete/Projects/self-improve/output/{version}/{dataset_name}/{mode}_{len(dataset)}_temperature_{temperature}_top-p_{top_p}.jsonl", "w") as f:
        for idx, result in enumerate(results):
            if isinstance(result, Exception):
                prediction = str(result)
            else:
                prediction = result["messages"][-1].content
            f.write(json.dumps({"question": dataset[idx]["question"], "answer": dataset[idx]["answer"], "prediction": prediction}) + "\n")

 30%|███       | 3/10 [06:26<16:03, 137.68s/it]/Users/ariete/Projects/self-improve/agent/utils/tools.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  result["title"] = BeautifulSoup(result["title"], "html.parser").get_text()
 50%|█████     | 5/10 [11:48<12:39, 151.95s/it]WARNING:root:Error response when hitting Wikibase: Error(code='label-not-defined', message='Item with the ID Q60997513 does not have a label in the language: en', additional_properties={})
/Users/ariete/Projects/self-improve/agent/utils/tools.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  result["title"] = BeautifulSoup(result["title"], "html.parser").get_text()
/opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParser

# HotpotQA

In [17]:
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
from langgraph.graph.graph import CompiledGraph


dataset_name = "ambig_qa"
mode = "react"
num_test_sample = 1000
temperature = 0
top_p = 1
batch_size = 100
llm = ChatOpenAI(temperature=temperature, top_p=top_p, model="gpt-4o-mini", base_url="https://api.chsdw.top/v1", max_retries=3)
graph: CompiledGraph = create_react_agent(llm, tools=tools, state_modifier=system_prompt)
from datasets import load_dataset
dataset: DatasetDict | Dataset | IterableDatasetDict | IterableDataset = load_dataset("json", data_files=f"../data/{dataset_name}.jsonl", split="train")
if num_test_sample > 0:
    dataset = dataset.select(range(num_test_sample))
print(dataset, dataset[0])

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 1000
}) {'question': 'Who was the man behind The Chipmunks?', 'question_id': 'tc_2', 'question_source': 'http://www.triviacountry.com/', 'entity_pages': {'doc_source': [], 'filename': [], 'title': [], 'wiki_context': []}, 'search_results': {'description': [], 'filename': [], 'rank': [], 'title': [], 'url': [], 'search_context': []}, 'answer': {'aliases': ['David Seville'], 'normalized_aliases': ['david seville'], 'matched_wiki_entity_name': '', 'normalized_matched_wiki_entity_name': '', 'normalized_value': 'david seville', 'type': 'WikipediaEntity', 'value': 'David Seville'}}


In [18]:
result = await react(item=dataset[0], reactor=graph)
print(result)

cannot access local variable 'input' where it is not associated with a value


In [16]:
results = []

In [ ]:
import asyncio
for i in tqdm(range(0, 1000, batch_size)):
    batch = dataset.select(range(i, min(i+batch_size, len(dataset))))
    results.extend(await asyncio.gather(*(react(item, reactor=graph) for item in batch)))
    with open(f"/Users/ariete/Projects/self-improve/output/inference/{dataset_name}/{mode}_{len(dataset)}_temperature_{temperature}_top-p_{top_p}.jsonl", "w") as f:
        for idx, result in enumerate(results):
            if isinstance(result, Exception):
                prediction = "None"
            else:
                prediction = result["messages"][-1].content
            f.write(json.dumps({"question": batch[idx]["question"], "answer": batch[idx]["answer"], "prediction": prediction}) + "\n")
